### RAS XS Geopackage Viewer

An interactive viewer to show the versioned and cataloged HEC-RAS XS

In [ ]:
import os
from pathlib import Path

from pyiceberg.catalog import load_catalog

from icefabric.helpers import load_creds, load_pyiceberg_config, to_geopandas

# Changes the current working dir to be the project root
current_working_dir = Path.cwd()
os.chdir(current_working_dir / "../../")
print(
    f"Changed current working dir from {current_working_dir} to: {Path.cwd()}. This notebook must run at the project root"
)

# dir is where the .env file is located
load_creds(dir=Path.cwd())

# Loading the local pyiceberg config settings
pyiceberg_config = load_pyiceberg_config(Path.cwd())
catalog = load_catalog(
    name="sql",
    type=pyiceberg_config["catalog"]["sql"]["type"],
    uri=pyiceberg_config["catalog"]["sql"]["uri"],
    warehouse=pyiceberg_config["catalog"]["sql"]["warehouse"],
)

Let's first load our tables that we'll be using and view the data schemas

In [ ]:
catalog.load_table("ras_xs.conflated")

Let's load a sample BLE dataset: "ble_05119_Pulaski". First we'll scan the pyiceberg catalog for all conflated XS within the BLE area, then pull the reference divides and flowpaths 

In [ ]:
from pyiceberg.expressions import EqualTo, In

gdf = to_geopandas(
    catalog.load_table("ras_xs.conflated")
    .scan(row_filter=EqualTo("domain", "/ble_05119_Pulaski"))
    .to_pandas()
)
gdf.head()

In [ ]:
reference_divides = to_geopandas(
    catalog.load_table("conus_reference.reference_divides")
    .scan(row_filter=In("flowpath_id", gdf["flowpath_id"]))
    .to_pandas()
)
reference_flowpaths = to_geopandas(
    catalog.load_table("conus_reference.reference_flowpaths")
    .scan(row_filter=In("flowpath_id", gdf["flowpath_id"]))
    .to_pandas()
)

In [ ]:
reference_divides = reference_divides.to_crs(epsg=3857)

reference_flowpaths = reference_flowpaths.to_crs(epsg=3857)
gdf = gdf.to_crs(epsg=3857)

ref_div_ex = reference_divides.explore(color="grey")
ref_flo_ex = reference_flowpaths.explore(m=ref_div_ex, color="blue")
gdf.explore(m=ref_flo_ex, color="black")

Now, we'll see how the representative XS look on this same data

In [ ]:
gdf_rep = to_geopandas(
    catalog.load_table("ras_xs.representative")
    .scan(row_filter=In("flowpath_id", gdf["flowpath_id"]))
    .to_pandas()
)

ref_div_ex = reference_divides.explore(color="grey")
ref_flo_ex = reference_flowpaths.explore(m=ref_div_ex, color="blue")
gdf_rep.explore(m=ref_flo_ex, color="red")

Let's now do the same work, but with a different area: "mip_18010110"

In [ ]:
gdf_mip = to_geopandas(
    catalog.load_table("ras_xs.conflated").scan(row_filter=EqualTo("domain", "/mip_18010110")).to_pandas()
)
gdf_mip.head()

In [ ]:
reference_divides_mip = to_geopandas(
    catalog.load_table("conus_reference.reference_divides")
    .scan(row_filter=In("flowpath_id", gdf_mip["flowpath_id"]))
    .to_pandas()
)
reference_flowpaths_mip = to_geopandas(
    catalog.load_table("conus_reference.reference_flowpaths")
    .scan(row_filter=In("flowpath_id", gdf_mip["flowpath_id"]))
    .to_pandas()
)
reference_divides_mip = reference_divides_mip.to_crs(epsg=3857)
reference_flowpaths_mip = reference_flowpaths_mip.to_crs(epsg=3857)
gdf_mip = gdf_mip.to_crs(epsg=3857)

ref_div_ex_mip = reference_divides_mip.explore(color="grey")
ref_flo_ex_mip = reference_flowpaths_mip.explore(m=ref_div_ex_mip, color="blue")
gdf_mip.explore(m=ref_flo_ex_mip, color="black")

Let's see how the representative XS look

In [ ]:
gdf_rep_mip = to_geopandas(
    catalog.load_table("ras_xs.representative")
    .scan(row_filter=In("flowpath_id", gdf_mip["flowpath_id"]))
    .to_pandas()
)

ref_div_ex_mip = reference_divides_mip.explore(color="grey")
ref_flo_ex_mip = reference_flowpaths_mip.explore(m=ref_div_ex_mip, color="blue")
gdf_rep_mip.explore(m=ref_flo_ex_mip, color="red")